# Prototype of Cast class

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import os
from shutil import copy
import pickle
import sys
import glob
import subprocess
import numpy as np

In [2]:
import pyPoseidon.model as pm
from pyPoseidon.utils import data

In [15]:
path='/Users/brey/Downloads/EUR/SCHISM/'

In [4]:
path='/Users/brey/Downloads/EUR/D3D/'

In [16]:
start_date = pd.to_datetime('2010-2-1 0:0:0')
end_date = pd.to_datetime('2010-2-2 0:0:0')
dates = pd.date_range(start_date,end_date, freq='12H')
dates

DatetimeIndex(['2010-02-01 00:00:00', '2010-02-01 12:00:00',
               '2010-02-02 00:00:00'],
              dtype='datetime64[ns]', freq='12H')

In [17]:
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in dates]
folders

['20100201.00', '20100201.12', '20100202.00']

In [18]:
meteo = []
PATH='/Users/brey/Downloads/'
for date in dates:
    end_date= pd.to_datetime(date) + pd.to_timedelta('12H')
    end_date = end_date.strftime(format='%Y-%m-%d %H:%M:%S')
    dr = pd.date_range(date, end_date, freq='12H')
    dur = [PATH+'{:04d}/{:02d}/{:02d}/'.format(x.year,x.month,x.day)+datetime.datetime.strftime(x, '%Y%m%d.%H')+'.tropical_cyclone.grib' for x in dr]
    meteo.append(dur)
meteo_files = meteo

In [19]:
meteo_files

[['/Users/brey/Downloads/2010/02/01/20100201.00.tropical_cyclone.grib',
  '/Users/brey/Downloads/2010/02/01/20100201.12.tropical_cyclone.grib'],
 ['/Users/brey/Downloads/2010/02/01/20100201.12.tropical_cyclone.grib',
  '/Users/brey/Downloads/2010/02/02/20100202.00.tropical_cyclone.grib'],
 ['/Users/brey/Downloads/2010/02/02/20100202.00.tropical_cyclone.grib',
  '/Users/brey/Downloads/2010/02/02/20100202.12.tropical_cyclone.grib']]

In [20]:
prev='20100201.00/'

In [21]:
#SCHISM
files = [ 'bctides.in', 'launchSchism.sh','sflux/sflux_inputs.txt']
files_sym = ['hgrid.gr3', 'hgrid.ll', 'manning.gr3', 'vgrid.in', 'drag.gr3', 'rough.gr3', 'windrot_geo2proj.gr3']
tag = 'eur'

In [50]:
#D3D                      
files=[tag+'_hydro.xml',tag+'.enc',tag+'.obs', tag+'.bnd', tag+'.bca','run_flow2d3d.sh']
files_sym=[tag+'.grd',tag+'.dep']
tag = 'eur01'

In [22]:
# load model
with open(path+prev+tag+'_info.pkl', 'r') as f:
                          info=pickle.load(f)



In [23]:
info

{'Tstart': 0,
 'Tstop': 720,
 'atm': True,
 'conda_env': 'mpi3',
 'config_file': '/Users/brey/SCHISM/test/param.in',
 'date': Timestamp('2010-02-01 00:00:00'),
 'dem': 'gebco',
 'dft': 1,
 'dpath': '/Users/brey/DATA/GEBCO_2014_2D.nc',
 'dt': 400,
 'end_date': Timestamp('2010-02-01 12:00:00'),
 'epath': '/Users/brey/SCHISM/v5.6.1/src/schism.ex_VL',
 'ft1': 0,
 'ft2': 11,
 'grid': 'tri2d',
 'grid_file': '/Users/brey/SCHISM/test/hgrid.gr3',
 'hgrid': '/Users/brey/SCHISM/test/hgrid.gr3',
 'maxlat': 75.79099261350001,
 'maxlon': 42.1752389468,
 'meteo': 'ecmwf_oper',
 'minlat': 25.024221636500002,
 'minlon': -35.9631540844,
 'mpaths': ['/Users/brey/Downloads/2010/02/01/20100201.00.tropical_cyclone.grib',
  '/Users/brey/Downloads/2010/02/01/20100201.12.tropical_cyclone.grib'],
 'ncores': 4,
 'params':                       vals
 attrs                     
 start_year            2010
 start_month              2
 start_day                1
 start_hour               0
 utc_start               0

In [24]:
kwargs={}
for date,folder,meteo in zip(dates[1:2],folders[1:2],meteo_files[1:2]):

    ppath = path+'/{}/'.format(prev) # mark the prev 
    
            
    prev = folder #set the new folder as prev 
    
    rpath=path+'/{}/'.format(folder)   # create the folder/run path

    #create if needed
    if not os.path.exists(rpath):  
                os.makedirs(rpath)

    copy(ppath+tag+'_info.pkl',rpath) #copy the info file from the previous path to the new rpath
    
    
    # load model
    with open(rpath+tag+'_info.pkl', 'r') as f:
                          info=pickle.load(f)

    
    args = set(kwargs.keys()).intersection(info.keys()) # modify dic with kwargs
    for attr in list(args):
            info[attr] = kwargs[attr]

    if info['solver']=='schism':
        info['config_file'] = ppath + 'param.in'
        #check for combine hotstart
        hotout=info['params'].loc['hotout_write'].values[0]
     
        resfiles=glob.glob(ppath+'/outputs/*_it*.nc')
        if not resfiles:
            ex=subprocess.Popen(args=['/Users/brey/SCHISM/v5.6.1/src/Utility/Combining_Scripts/combine_hotstart7 -i {}'.format(hotout)], cwd=ppath+'/outputs/', shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE, bufsize=1)

            for line in iter(ex.stdout.readline,b''): 
                sys.stdout.write(line)
                sys.stdout.flush()  
    elif info['solver']=='d3d':
        info['config_file'] = ppath + info['tag']+'.mdf'

    #update the properties              
    info['date'] = date
    info['start_date'] = date
    info['end_date'] = date + pd.to_timedelta(info['time_frame'])
    info['mpaths'] = meteo
    info['rpath'] = rpath

        
    m=pm.model(**info)

    
    
    #D3D
    if info['solver']=='d3d':
        inresfile='tri-rst.'+m.impl.tag+'.'+datetime.datetime.strftime(date,'%Y%m%d.%H%M%M')
        outresfile='restart.'+ datetime.datetime.strftime(date,'%Y%m%d.%H%M%M')

        m.impl.config(output=True,config={'Restid ': outresfile}) 
    
        outresfile = 'tri-rst.' + outresfile 


    #SCHISM
    elif info['solver']=='schism':
            #Restart File  SCHISM           
        inresfile='/outputs/hotstart_it={}.nc'.format(hotout)
        outresfile='/hotstart.nc'
        m.impl.config(output=True,config={'ihot': 1, 'nramp_elev' : 1})


    
    # copy/link necessary files
    for filename in files:
        ipath = glob.glob(ppath+filename)
        if ipath:
            try:
                copy(ppath+filename,rpath+filename)
            except:
                dir_name ,file_name = os.path.split(filename)
                if not os.path.exists(rpath + dir_name):
                    os.makedirs(rpath + dir_name)
                copy(ppath+filename,rpath+filename)


    
    #symlink the big files
    for filename in files_sym:
        ipath = glob.glob(path+folders[0]+'/'+filename)
        if ipath:
            try:
                os.symlink(ipath[0],rpath+filename)
            except OSError, e:
                if e.errno == os.errno.EEXIST:
                    sys.stdout.write('Restart link present\n')
                    sys.stdout.write('overwriting\n')
                    os.remove(rpath+filename)
                    os.symlink(ipath[0],rpath+filename)


    sys.stdout.write('set restart\n')
    sys.stdout.flush()
        
    try:
        os.symlink(ppath+inresfile,rpath+outresfile)
    except OSError, e:
        if e.errno == os.errno.EEXIST:
            sys.stdout.write('Restart link present\n')
            sys.stdout.write('overwriting\n')
            os.remove(rpath+outresfile)
            os.symlink(ppath+inresfile,rpath+outresfile)
        else:
            raise e            
          
                    
    #get new meteo 

    sys.stdout.write('process meteo\n')
    sys.stdout.flush()

            
    m.force()
    m.impl.to_force(m.impl.meteo.impl.uvp,vars=['msl','u10','v10'],rpath=rpath)  #write u,v,p files 
    

    
    
     # run case
    sys.stdout.write('executing\n')
    sys.stdout.flush()
        
        
    m.save()

    m.run()



Restart link present
overwriting
Restart link present
overwriting
Restart link present
overwriting
Restart link present
overwriting
Restart link present
overwriting
set restart
process meteo

extracting meteo from /Users/brey/Downloads/2010/02/01/20100201.12.tropical_cyclone.grib

retrieving msl at 2010-02-01 12:00:00

retrieving 10u at 2010-02-01 12:00:00

retrieving 10v at 2010-02-01 12:00:00

retrieving msl at 2010-02-01 15:00:00

retrieving 10u at 2010-02-01 15:00:00

retrieving 10v at 2010-02-01 15:00:00

retrieving msl at 2010-02-01 18:00:00

retrieving 10u at 2010-02-01 18:00:00

retrieving 10v at 2010-02-01 18:00:00

retrieving msl at 2010-02-01 21:00:00

retrieving 10u at 2010-02-01 21:00:00

retrieving 10v at 2010-02-01 21:00:00

extracting meteo from /Users/brey/Downloads/2010/02/02/20100202.00.tropical_cyclone.grib

retrieving msl at 2010-02-02 00:00:00

retrieving 10u at 2010-02-02 00:00:00

retrieving 10v at 2010-02-02 00:00:00

retrieving msl at 2010-02-02 03:00:00

retr

## Check output for D3D

### Unified run

In [25]:
out0 = data(**{'solver':'d3d','rpath': '/Users/brey/Downloads/EUR/D3D/20100201.00_test/'})


reading data based on /Users/brey/Downloads/EUR/D3D/20100201.00_test/eur01_info.pkl 


In [26]:
out0.impl.Dataset

<xarray.Dataset>
Dimensions:  (XU: 769, XV: 769, XZ: 769, YU: 514, YV: 514, YZ: 514, time: 25)
Coordinates:
  * YU       (YU) float64 25.15 25.25 25.35 25.45 25.55 25.65 25.75 25.85 ...
  * XZ       (XZ) float64 -34.95 -34.85 -34.75 -34.65 -34.55 -34.45 -34.35 ...
  * YZ       (YZ) float64 25.1 25.2 25.3 25.4 25.5 25.6 25.7 25.8 25.9 26.0 ...
  * time     (time) datetime64[ns] 2010-02-01 2010-02-01T01:00:00 ...
  * XU       (XU) float64 -34.95 -34.85 -34.75 -34.65 -34.55 -34.45 -34.35 ...
  * XV       (XV) float64 -34.9 -34.8 -34.7 -34.6 -34.5 -34.4 -34.3 -34.2 ...
  * YV       (YV) float64 25.1 25.2 25.3 25.4 25.5 25.6 25.7 25.8 25.9 26.0 ...
Data variables:
    V1Z      (time, YZ, XZ) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    V1       (time, YV, XV) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    U1Z      (time, YZ, XZ) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    PATM     (time, YZ, XZ) float32 101386.68 101375.99 101364.4 101352.4 ...
    S1       (

### Split runs

#### 0-12 H

In [27]:
out1 = data(**{'solver':'d3d','rpath': '/Users/brey/Downloads/EUR/D3D/20100201.00/'})


reading data based on /Users/brey/Downloads/EUR/D3D/20100201.00/eur01_info.pkl 


Mask values to avoid the comparison with the NaNs

In [28]:
#First Part
ma0=np.ma.masked_invalid(out0.impl.Dataset.S1.values[:13,:])
ma1=np.ma.masked_invalid(out1.impl.Dataset.S1.values)

In [29]:
ma0.shape,ma1.shape

((13, 514, 769), (13, 514, 769))

In [30]:
np.array_equal(ma0,ma1)

False

In [31]:
np.max(np.abs(ma0-ma1))

0.0

#### 12-24 H

In [32]:
out2 = data(**{'solver':'d3d','rpath': '/Users/brey/Downloads/EUR/D3D/20100201.12/'})


reading data based on /Users/brey/Downloads/EUR/D3D/20100201.12/eur01_info.pkl 


In [33]:
#Second part
ma02=np.ma.masked_invalid(out0.impl.Dataset.S1.values[12:,:])
ma2=np.ma.masked_invalid(out2.impl.Dataset.S1.values)

In [34]:
np.array_equal(ma02,ma2)

False

In [35]:
np.max(np.abs(ma02-ma2))

1.1920929e-07

## Check output for SCHISM

### Unified run

In [36]:
out0 = data(**{'solver':'schism','rpath': '/Users/brey/Downloads/EUR/SCHISM/20100201.00_test/','savenc' : True})

Combining output


In [37]:
out0.impl.Dataset.time

<xarray.DataArray 'time' (time: 24)>
array(['2010-02-01T01:00:00.000000000', '2010-02-01T02:00:00.000000000',
       '2010-02-01T03:00:00.000000000', '2010-02-01T04:00:00.000000000',
       '2010-02-01T05:00:00.000000000', '2010-02-01T06:00:00.000000000',
       '2010-02-01T07:00:00.000000000', '2010-02-01T08:00:00.000000000',
       '2010-02-01T09:00:00.000000000', '2010-02-01T10:00:00.000000000',
       '2010-02-01T11:00:00.000000000', '2010-02-01T12:00:00.000000000',
       '2010-02-01T13:00:00.000000000', '2010-02-01T14:00:00.000000000',
       '2010-02-01T15:00:00.000000000', '2010-02-01T16:00:00.000000000',
       '2010-02-01T17:00:00.000000000', '2010-02-01T18:00:00.000000000',
       '2010-02-01T19:00:00.000000000', '2010-02-01T20:00:00.000000000',
       '2010-02-01T21:00:00.000000000', '2010-02-01T22:00:00.000000000',
       '2010-02-01T23:00:00.000000000', '2010-02-02T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2010-02-

### Split runs

#### 0-12 H

In [111]:
out1 = data(**{'solver':'schism','rpath': '/Users/brey/Downloads/EUR/SCHISM/20100201.00/', 'savenc' : True})

Combining output


In [39]:
out1.impl.Dataset.time

<xarray.DataArray 'time' (time: 12)>
array(['2010-02-01T01:00:00.000000000', '2010-02-01T02:00:00.000000000',
       '2010-02-01T03:00:00.000000000', '2010-02-01T04:00:00.000000000',
       '2010-02-01T05:00:00.000000000', '2010-02-01T06:00:00.000000000',
       '2010-02-01T07:00:00.000000000', '2010-02-01T08:00:00.000000000',
       '2010-02-01T09:00:00.000000000', '2010-02-01T10:00:00.000000000',
       '2010-02-01T11:00:00.000000000', '2010-02-01T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2010-02-01T01:00:00 2010-02-01T02:00:00 ...

In [40]:
#ELEV
np.array_equal(out0.impl.Dataset.elev.values[:12,:],out1.impl.Dataset.elev.values)

True

In [41]:
#WIND
np.array_equal(out0.impl.Dataset.wind_speed.values[:12,:,0],out1.impl.Dataset.wind_speed.values[:,:,0])

True

#### 12-24 H

In [106]:
out2 = data(**{'solver':'schism','rpath': '/Users/brey/Downloads/EUR/SCHISM/20100201.12/', 'savenc' : True})

Combining output


In [107]:
out2.impl.Dataset.time

<xarray.DataArray 'time' (time: 12)>
array(['2010-02-01T13:00:00.000000000', '2010-02-01T14:00:00.000000000',
       '2010-02-01T15:00:00.000000000', '2010-02-01T16:00:00.000000000',
       '2010-02-01T17:00:00.000000000', '2010-02-01T18:00:00.000000000',
       '2010-02-01T19:00:00.000000000', '2010-02-01T20:00:00.000000000',
       '2010-02-01T21:00:00.000000000', '2010-02-01T22:00:00.000000000',
       '2010-02-01T23:00:00.000000000', '2010-02-02T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2010-02-01T13:00:00 2010-02-01T14:00:00 ...

In [108]:
out0.impl.Dataset.elev.values[12:,:].shape, out2.impl.Dataset.elev.values.shape

((12, 175156), (12, 175156))

In [109]:
#Elev
np.array_equal(out0.impl.Dataset.elev.values[12:,:],out2.impl.Dataset.elev.values)

False

In [110]:
np.max(np.abs(out0.impl.Dataset.elev.values[12:,:] - out2.impl.Dataset.elev.values))

0.09468653

In [83]:
out0.impl.Dataset.time.values[12] , out2.impl.Dataset.time.values[0]

(numpy.datetime64('2010-02-01T13:00:00.000000000'),
 numpy.datetime64('2010-02-01T13:00:00.000000000'))

In [84]:
out0.impl.Dataset.elev.values[12,:], out2.impl.Dataset.elev.values[0,:]

(array([-0.02433086, -0.02383262, -0.02320769, ..., -0.00485766,
        -0.00341343, -0.00402453], dtype=float32),
 array([-0.02449196, -0.02387583, -0.02311712, ..., -0.00996716,
        -0.00859527, -0.00924963], dtype=float32))

In [85]:
#WIND
np.array_equal(out0.impl.Dataset.wind_speed.values[12:,:], out2.impl.Dataset.wind_speed.values)

True

#### check the restart file

In [51]:
res = xr.open_dataset('/Users/brey/Downloads/EUR/SCHISM/20100201.12/hotstart.nc')

In [52]:
res

<xarray.Dataset>
Dimensions:  (elem: 339138, nVert: 2, node: 175156, ntracers: 2, one_new: 1, side: 514457)
Dimensions without coordinates: elem, nVert, node, ntracers, one_new, side
Data variables:
    idry_e   (elem) int32 ...
    idry_s   (side) int32 ...
    idry     (node) int32 ...
    eta2     (node) float64 ...
    we       (elem, nVert) float64 ...
    tr_el    (elem, nVert, ntracers) float64 ...
    su2      (side, nVert) float64 ...
    sv2      (side, nVert) float64 ...
    tr_nd    (node, nVert, ntracers) float64 ...
    tr_nd0   (node, nVert, ntracers) float64 ...
    q2       (node, nVert) float64 ...
    xl       (node, nVert) float64 ...
    dfv      (node, nVert) float64 ...
    dfh      (node, nVert) float64 ...
    dfq1     (node, nVert) float64 ...
    dfq2     (node, nVert) float64 ...
    time     (one_new) float64 ...
    iths     (one_new) int32 ...
    ifile    (one_new) int32 ...

In [54]:
108*400/3600

12

In [66]:
res.eta2.values.shape, out1.impl.Dataset.elev.values[-1,:].shape

((175156,), (175156,))

In [65]:
np.array_equal(res.eta2.values,out1.impl.Dataset.elev.values[-1,:])

False

In [67]:
np.max(np.abs(res.eta2.values-out1.impl.Dataset.elev.values[-1,:]))

7.449911132439979e-09

In [58]:
res.time

<xarray.DataArray 'time' (one_new: 1)>
array([43200.])
Dimensions without coordinates: one_new

In [61]:
out1.impl.Dataset.elev.values[-1,:]

array([-0.0408535 , -0.04038654, -0.03975742, ...,  0.00247313,
        0.00420236,  0.00347213], dtype=float32)

In [104]:
res.su2.values

array([[ 0.00022945,  0.00022945],
       [ 0.00029706,  0.00029706],
       [ 0.0004761 ,  0.0004761 ],
       ...,
       [-0.00318269, -0.00318269],
       [-0.00355651, -0.00355651],
       [-0.00237146, -0.00237146]])